In [1]:
from mastodon import Mastodon
from datetime import datetime
import pytz

import os
from dotenv import load_dotenv

import requests

load_dotenv()

True

In [2]:
mastodon = Mastodon(
    client_id=os.environ.get("MASTODON_CLIENT_KEY"),
    client_secret=os.environ.get("MASTODON_CLIENT_SECRET"),
    access_token=os.environ.get("MASTODON_ACCESS_TOKEN"),
    api_base_url=os.environ.get("MASTODON_URL_SOCIAL")
)

In [ ]:
mastodon.timeline_public(min_id='102884194951174805', limit=1)

In [72]:
def binary_search_date(since_date):
    lower_id = 82884194951174805
    upper_id = 102884194951174805

    while True:
        if lower_id is None:
            toot = mastodon.timeline_public(limit=1)[0]
        else:
            toot = mastodon.timeline_public(min_id=lower_id, limit=1)[0]

        created_at = toot['created_at']
        if created_at < since_date:
            lower_id = toot['id']
        else:
            upper_id = toot['id']
            break

    while upper_id - lower_id > 1:
        mid_id = (upper_id + lower_id) // 2
        toot = mastodon.timeline_public(max_id=mid_id, limit=1)[0]
        created_at = toot['created_at']
        if created_at < since_date:
            lower_id = toot['id']
        else:
            upper_id = toot['id']
        
        print(f'\rCurrent binary search status: lower_id = {lower_id}, upper_id = {upper_id}', end='')

    print()  # To move to the next line in the terminal after the search is completed
    return lower_id, upper_id

In [73]:
binary_search_date(since_date)

KeyboardInterrupt: 

In [ ]:
def fetch_historical_data_with_limit(since_date, until_date, fetch_limit=1000, toots_per_request=40, daily_limit=5000):
    all_toots = []
    fetched_count = 0
    max_tries = daily_limit // toots_per_request
    pause_interval = 100

    while fetched_count < daily_limit:
        remaining_limit = daily_limit - fetched_count
        current_fetch_limit = min(fetch_limit, remaining_limit)
        fetched_toots = fetch_historical_data(since_date, until_date, current_fetch_limit, toots_per_request)
        all_toots.extend(fetched_toots)
        fetched_count += len(fetched_toots)

        if fetched_count >= daily_limit or len(fetched_toots) < current_fetch_limit:
            break

        if fetched_count % (toots_per_request * pause_interval) == 0:
            time.sleep(600)  # Wait for 10 minutes before continuing to fetch data

    return all_toots

In [37]:
fetched_toots = mastodon.timeline_public(max_id=None, limit=10)
fetched_toots

[{'id': 110327632389558462,
  'created_at': datetime.datetime(2023, 5, 7, 13, 27, 47, tzinfo=tzutc()),
  'in_reply_to_id': None,
  'in_reply_to_account_id': None,
  'sensitive': False,
  'spoiler_text': '',
  'visibility': 'public',
  'language': 'en',
  'uri': 'https://freeradical.zone/users/kmontenegro/statuses/110327632185525624',
  'url': 'https://freeradical.zone/@kmontenegro/110327632185525624',
  'replies_count': 0,
  'reblogs_count': 0,
  'favourites_count': 0,
  'edited_at': None,
  'favourited': False,
  'reblogged': False,
  'muted': False,
  'bookmarked': False,
  'content': '<p>The rising Republican movement to defund public libraries<br><a href="https://artifact.news/s/-giWM1SYP2o" rel="nofollow noopener noreferrer" target="_blank"><span class="invisible">https://</span><span class="">artifact.news/s/-giWM1SYP2o</span><span class="invisible"></span></a>=</p>',
  'filtered': [],
  'reblog': None,
  'account': {'id': 792856,
   'username': 'kmontenegro',
   'acct': 'kmonten

In [27]:
def fetch_historical_data(since_date, until_date, fetch_limit=10, toots_per_request=50):
    all_toots = []
    fetched_count = 0

    since_id = mastodon.timeline_public(since=since_date, limit=1)[0]['id']
    until_id = mastodon.timeline_public(until=until_date, limit=1)[0]['id']
    
    max_id = until_id

    while fetched_count < fetch_limit:
        remaining_limit = fetch_limit - fetched_count
        limit = min(remaining_limit, toots_per_request)
        
        fetched_toots = mastodon.timeline_public(max_id=max_id, since_id=since_id, limit=limit)
        if not fetched_toots:
            break

        all_toots.extend(fetched_toots)
        fetched_count += len(fetched_toots)

        max_id = fetched_toots[-1]['id']

    return all_toots

In [30]:
timezone = pytz.UTC
since_date = datetime(2023, 1, 1, tzinfo=timezone)  # Fetch data since January 1, 2020
until_date = datetime(2023, 1, 1, tzinfo=timezone)  # Fetch data until Januray 1, 2020

In [31]:
historical_data = fetch_historical_data(since_date, until_date, fetch_limit=10, toots_per_request=50)

KeyboardInterrupt: 

In [ ]:
# print the fetched historical data
for toot in historical_data:
    print(toot['content'])